In [1]:
import os
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib import cm

import ipywidgets as widgets
from ipywidgets import interactive, interact, interact_manual
from IPython.display import Image

# Lettura csv degli esperimenti
df = pd.read_csv('./experiments_collector.csv', parse_dates=["Start Date", "End Date"])

In [2]:
def plot_data(data, experiment):
    DT=data['datetime']
    V=data['voltage']
    R=data['resistance']
    T=data['temperature']
    I=data['current_source']
    J=data['current_density']
    E=data['electric_field']
    RHO=data['resistivity']
    
    ## Electrical field, Resistivity
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=DT, y=E, name="Electrical Field"), 
        secondary_y=False,
    )

    
    fig.add_trace(
        go.Scatter(x=DT, y=RHO, name="Resistivity"),
        secondary_y=True 
    )

    fig.update_traces(mode="lines+markers")
    
    # Add figure title
    fig.update_layout(
        title_text="Electrical field, Resistivity"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Time")

    # Set y-axes titles
    fig.update_yaxes(title_text="E [V/cm]", secondary_y=False)
    fig.update_yaxes(title_text="RHO [Ohm cm]", secondary_y=True)
    fig.show()
    
    ## Current Density, Temperature
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=DT, y=J, name="Current Density"),
        secondary_y=False,
    )

    
    fig.add_trace(
        go.Scatter(x=DT, y=T, name="Temperature"),
        secondary_y=True
    )

    fig.update_traces(mode="lines+markers")

    # Add figure title
    fig.update_layout(
        title_text="Current Density, Temperature"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Time")

    # Set y-axes titles
    fig.update_yaxes(title_text="J [A/cm2]", secondary_y=False)
    fig.update_yaxes(title_text="T [°K]", secondary_y=True)
    fig.show()
    
    if np.min(I) != np.max(I) and 'square' not in experiment:
            max_index = np.argmax(I)
            fig = px.line(data, x=J[0:max_index], y=E[0:max_index], labels={"x":'J [A/cm2]', "y":'E [V/cm]'},
            title='Current density vs Electrical density',
            log_x=True,  log_y=True,
            markers=True,

            #width=952, height=800
            )
            fig.show()



# Experiment Plotter

In [3]:
@interact
def show_experiment(experiment=df['Experiment']):
    name = df.loc[df['Experiment'] == experiment]['Name'].values[0]
    rxd_path = '../' + name + '/' + name + '_Info/' + name + '-diffrattogramma.png'
    readme_path = '../' + name + '/README'
    my_name = name
    try:
        # README
        with open(readme_path, "r", encoding='utf-8') as file_desc:
            text = file_desc.read()
            print(text)
    except FileNotFoundError:
        print(name)
    experiment_dir = experiment[:-15]   
    if experiment[-2:] == '_a':
        experiment_dir = experiment[:-17]
    elif '-joined' in experiment:
        experiment = experiment.replace('-joined', '')
        experiment_dir = experiment[:-15]
    print(experiment_dir)
    file_path = '../' + name + '/' + experiment_dir  + '/' + experiment
    # File di descrizione
    with open(file_path, "r", encoding='utf-8') as file_desc:
        text = file_desc.read()
        print(text)
    if os.path.exists(rxd_path):
        display(Image(rxd_path))
    
#    df_data = pd.read_csv(file_path + '.csv')
#    print(df_data)

    data = np.load(file_path + '.npz', allow_pickle=True)
  
    plot_data(data, experiment)


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.
